# <div align="center" style="color: #ff5733;">Data Requested By Annastasia for Monthly Reporting</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
# Year and Month for which data is extracted
data_year = 2025
data_month = 1

In [3]:
# How many Applications applied to each month

sq = """
# loan applied
SELECT 
    DATE_TRUNC( startApplyDateTime,MONTH) as mm, 
    case when reloan_flag = 1  and loantype not like 'FLEXUP' then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
         
         else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as StartedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1 desc,2;
"""

StartedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in StartedApps are:\t {StartedApps.shape}")

Job ID 94a9315c-dee7-4c74-8454-794c698f725b successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in StartedApps are:	 (242, 3)


In [4]:
# How many submitted applications to each month
sq = """
# loan Submitted
SELECT 
    DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime else termsAndConditionsSubmitDateTime end),MONTH) as mm,  
    case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as SubmittedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
group by 1,2
order by 1 desc,2;
"""

SubmittedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns of SubmittedApps are:\t {SubmittedApps.shape}")
SubmittedApps.sample(10)

Job ID 9383ad4e-a043-4b21-a895-2313df284a2f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns of SubmittedApps are:	 (240, 3)


,mm,LoanProduct,SubmittedApps
148,2023-03-01,Reloan,174
40,2024-09-01,SIL-Instore,21930
47,2024-08-01,SIL-Instore,24161
10,2025-01-01,Reloan,1119
167,2022-11-01,Big Loan,4
29,2024-11-01,SIL ZERO,2296
9,2025-01-01,Quick,41196
175,2022-10-01,SIL-Instore,187
146,2023-03-01,Flex-up,5
170,2022-11-01,SIL-Instore,260


In [5]:
# Approved Loans for each month

sq = """
# Approved loans
SELECT 
DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime 
                  when reloan_flag = 1 and loantype not like 'FLEXUP' then startApplyDateTime else decision_date end),MONTH) as mm,
        case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as ApprovedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
where 
(case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then flagDisbursement 
      when reloan_flag = 1 and loantype not like 'FLEXUP' then flagDisbursement  else flagApproval end) = 1
group by 1,2
order by 1 desc,2
;
"""

ApprovedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in ApprovedApps are:\t {ApprovedApps.shape}")
ApprovedApps.head(10)

Job ID 55f2cbfb-83f3-4748-af4d-004bfd0fe5d1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in ApprovedApps are:	 (204, 3)


,mm,LoanProduct,ApprovedApps
0,2025-02-01,ACL TSA,145
1,2025-02-01,Flex-up,96
2,2025-02-01,Quick,1083
3,2025-02-01,Reloan,283
4,2025-02-01,SIL Competitor,426
5,2025-02-01,SIL ZERO,275
6,2025-02-01,SIL-Instore,3952
7,2025-01-01,ACL TSA,358
8,2025-01-01,Flex-up,461
9,2025-01-01,Quick,3585


In [6]:
# Booked Loans for each month

sq = """
#Booked loans
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
            case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as BookedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where flagDisbursement = 1

group by 1,2
order by 1 desc,2
;
"""

BookedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedApps are:\t {BookedApps.shape}")
BookedApps.head(10)

Job ID 79f52e4e-fe9d-4fa0-b80d-8aaa2d926768 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedApps are:	 (199, 3)


,mm,LoanProduct,BookedApps
0,2025-02-01,ACL TSA,139
1,2025-02-01,Flex-up,167
2,2025-02-01,Quick,1068
3,2025-02-01,Reloan,283
4,2025-02-01,SIL Competitor,422
5,2025-02-01,SIL ZERO,265
6,2025-02-01,SIL-Instore,3880
7,2025-01-01,ACL TSA,348
8,2025-01-01,Flex-up,605
9,2025-01-01,Quick,3509


In [7]:
# Get the total booked amount for a month

sq = """
# Booked Amount
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    sum (disbursedLoanAmount) as BookedAmt
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
     where flagDisbursement = 1
group by 1,2
order by 1 desc,2
;
"""
BookedAmt = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedAmt are:\t {BookedAmt.shape}")
BookedAmt.head(10)

Job ID 37a12f96-6734-4e70-a3b8-db299df229d3 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedAmt are:	 (199, 3)


,mm,LoanProduct,BookedAmt
0,2025-02-01,ACL TSA,695000.00
1,2025-02-01,Flex-up,4693797.04
2,2025-02-01,Quick,16431750.00
3,2025-02-01,Reloan,4995437.02
4,2025-02-01,SIL Competitor,5517371.00
5,2025-02-01,SIL ZERO,4374203.00
6,2025-02-01,SIL-Instore,39672461.00
7,2025-01-01,ACL TSA,1740000.00
8,2025-01-01,Flex-up,17824415.24
9,2025-01-01,Quick,54034155.00


In [8]:
BookedAmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   mm           199 non-null    datetime64[us]
 1   LoanProduct  199 non-null    object        
 2   BookedAmt    199 non-null    float64       
dtypes: datetime64[us](1), float64(1), object(1)
memory usage: 4.8+ KB


In [9]:
raw_data = StartedApps.merge(SubmittedApps,how='outer').merge(ApprovedApps,how='outer').merge(BookedApps,how='outer').merge(BookedAmt,how='outer')

In [10]:
raw_data.tail(10)

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
234,2025-01-01,SIL Repeat,435,429,363,360,5.004474e+06
235,2025-01-01,SIL ZERO,763,758,423,417,6.627350e+06
236,2025-01-01,SIL-Instore,21459,21217,12240,12076,1.281947e+08
237,2025-02-01,ACL TSA,244,192,145,139,6.950000e+05
238,2025-02-01,Flex-up,96,96,96,167,4.693797e+06
239,2025-02-01,Quick,18262,15249,1083,1068,1.643175e+07
240,2025-02-01,Reloan,330,307,283,283,4.995437e+06
241,2025-02-01,SIL Competitor,658,516,426,422,5.517371e+06
242,2025-02-01,SIL ZERO,467,462,275,265,4.374203e+06
243,2025-02-01,SIL-Instore,6726,6650,3952,3880,3.967246e+07


In [11]:
print(f'{data_year}-{data_month:02d}-01')

2025-01-01


In [12]:
raw_data_current_month = raw_data[(raw_data['mm']>='2022-08-01') &(raw_data['mm']<=f'{data_year}-{data_month:02d}-01')]

In [13]:
raw_data_current_month

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
56,2022-08-01,Flex,5558,4732,596,599,1.198500e+07
57,2022-08-01,Quick,43764,30660,1929,1926,3.300100e+07
58,2022-08-01,SIL-Instore,967,956,459,457,1.137941e+07
59,2022-08-01,SIL-Online(discontinued),1192,652,43,41,1.197499e+06
60,2022-09-01,Big Loan,13,12,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...
232,2025-01-01,Reloan,1151,1119,1072,1072,1.936355e+07
233,2025-01-01,SIL Competitor,1391,1072,906,897,1.237838e+07
234,2025-01-01,SIL Repeat,435,429,363,360,5.004474e+06
235,2025-01-01,SIL ZERO,763,758,423,417,6.627350e+06


### Loan Account Tagging

In [14]:
sq="""
# Loan Tagging
SELECT
    loanAccountNumber,
    Case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    disbursementDateTime,
    (CASE WHEN new_loan_type = 'Flex-up' then
    LAG(new_loan_type) OVER (PARTITION BY customerId ORDER BY disbursementDateTime) END) AS OriginalLoanProduct
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
   where flagDisbursement=1
;
"""
loan_type = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in loan_type are: {loan_type.shape}")
loan_type.tail(10)

Job ID 3f5ff20e-f422-43e1-ae99-8e735117eeb8 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in loan_type are: (306821, 4)


,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
306811,60832462150012,SIL-Instore,2025-02-08 17:54:48,None
306812,60832473900011,SIL-Instore,2025-02-09 12:06:33,None
306813,60832475590017,SIL-Instore,2025-02-09 12:55:12,None
306814,60832476750011,SIL-Instore,2025-02-09 13:21:43,None
306815,60832479500011,SIL-Instore,2025-02-09 14:40:11,None
306816,60832480220013,SIL-Instore,2025-02-09 15:10:45,None
306817,60832483720012,SIL-Instore,2025-02-09 16:29:09,None
306818,60832488860013,SIL-Instore,2025-02-09 19:28:16,None
306819,60832499300014,SIL-Instore,2025-02-10 12:11:53,None
306820,60832506940011,SIL-Instore,2025-02-10 16:50:08,None


In [15]:
loan_type[loan_type['loanAccountNumber']=='60828136490016']

,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
231160,60828136490016,SIL-Instore,2024-09-01 12:25:12,None


In [16]:
Loan_Acs_current_month = loan_type[(loan_type['disbursementDateTime'].dt.month == data_month) & (loan_type['disbursementDateTime'].dt.year == data_year)]

In [17]:
Loan_Acs_current_month = Loan_Acs_current_month[['loanAccountNumber', 'LoanProduct','OriginalLoanProduct']]

#### VAS

In [18]:
sq = """
# Vas 
SELECT 
    extract(year from disbursementDateTime) as year , 
    extract(month from disbursementDateTime) as month,
    Case when reloan_flag = 1 and b.loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when b.loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as new_loan_type, 
    count(distinct  b.loanAccountNumber ) AS TotalSold,
   
   count(distinct CASE WHEN vas_flag = 'true' THEN b.loanAccountNumber END) AS SoldWithVAS
     FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a join
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` b on a.loanAccountNumber = b.loanAccountNumber
where flagDisbursement = 1
group by 1,2 ,3 
order by 1 desc,2 desc,3
;
"""

vas_df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in vas_df are:\t {vas_df.shape}")
vas_df.head(10)

Job ID 371284f8-ce3d-43c3-96b3-69bae56930c1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in vas_df are:	 (199, 5)


,year,month,new_loan_type,TotalSold,SoldWithVAS
0,2025,2,ACL TSA,139,33
1,2025,2,Flex-up,167,76
2,2025,2,Quick,1068,536
3,2025,2,Reloan,283,132
4,2025,2,SIL Competitor,422,262
5,2025,2,SIL ZERO,265,156
6,2025,2,SIL-Instore,3880,2764
7,2025,1,ACL TSA,348,74
8,2025,1,Flex-up,605,245
9,2025,1,Quick,3509,1764


In [19]:
vas_df_current_month = vas_df[(vas_df['year']==data_year)& (vas_df['month']==data_month)]

In [20]:
with pd.ExcelWriter(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v1.xlsx') as writer:
    # Write each DataFrame to a specific sheet in the Excel file
    raw_data_current_month.to_excel(writer, sheet_name='raw_data', index=False)
    Loan_Acs_current_month.to_excel(writer, sheet_name='Loan Acs', index=False)
    vas_df_current_month.to_excel(writer, sheet_name='VAS', index=False)

In [21]:
print(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v1.xlsx')

loan_accounts_tagging_2025_01_v1.xlsx
